In [1]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

In [2]:
def randomForest():
    bist100 = pd.read_csv("/content/drive/MyDrive/dp/SP500.csv")
    bist100.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close"},
                   inplace=True)
    bist100.isnull().sum()
    bist100.isna().any()
    bist100.dropna(inplace=True)
    bist100.isna().any()
    bist100['date'] = pd.to_datetime(bist100.date)
    bist100.sort_values(by='date', inplace=True)
    print("*******************************************************************************************")
    print("Starting date: ", bist100.iloc[0][0])
    print("Ending date: ", bist100.iloc[-1][0])
    print("Duration: ", bist100.iloc[-1][0] - bist100.iloc[0][0])
    bist100.groupby(bist100['date'].dt.strftime('%B'))['low'].min()
    closedf = bist100[['date', 'close']]
    close_stock = closedf.copy()
    del closedf['date']
    scaler = MinMaxScaler(feature_range=(0, 1))
    closedf = scaler.fit_transform(np.array(closedf))
    training_size = int(len(closedf) * 0.65)
    test_size = len(closedf) - training_size
    train_data, test_data = closedf[0:test_size, :], closedf[test_size:len(closedf), :]
    def create_dataset(dataset, time_step=5):
        dataX, dataY = [], []
        for i in range(len(dataset) - time_step - 1):
            a = dataset[i:(i + time_step), 0] 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)

    time_step = 10
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(max_depth=1)
    regressor.fit(X_train, y_train)
    train_predict = regressor.predict(X_train)
    test_predict = regressor.predict(X_test)
    train_predict = train_predict.reshape(-1, 1)
    test_predict = test_predict.reshape(-1, 1)
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)
    original_ytrain = scaler.inverse_transform(y_train.reshape(-1, 1))
    original_ytest = scaler.inverse_transform(y_test.reshape(-1, 1))
    print("Random forest MSE: ", mean_squared_error(original_ytest, test_predict))
    print("*******************************************************************************************\n\n\n\n")


In [3]:
import json
import time
import math
from sklearn.ensemble import RandomForestClassifier as RFC
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

def singleInputLSTM():
    split = (0.65)
    sequence_length = 10;
    normalise = True
    batch_size = 100;
    input_dim = 1
    input_timesteps = 9
    neurons = 50
    epochs = 5
    prediction_len = 1
    dense_output = 1
    drop_out = 0
    dataframe = pd.read_csv("/content/drive/MyDrive/dp/source_price.csv")
    cols = ['Adj Close']
    len_dataframe = dataframe.shape[0]
    i_split = int(len(dataframe) * split)
    data_train = dataframe.get(cols).values[:i_split]
    data_test = dataframe.get(cols).values[i_split:]
    len_train = len(data_train)
    len_test = len(data_test)
    len_train_windows = None

    data_windows = []
    for i in range(len_test - sequence_length):
        data_windows.append(data_test[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    y_test_ori = data_windows[:, -1, [0]]
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2] 
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num): 
        normalised_window = []
        for col_i in range(0, col_num):
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min)
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)

    data_windows = normalised_data 
    x_test = data_windows[:, :-1]
    y_test = data_windows[:, -1, [0]]

    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    for win_i in range(0, win_num): 
        normalised_window = []
        for col_i in range(0, col_num):
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train = data_windows[:, :-1]
    y_train = data_windows[:, -1, [0]]

    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences=True))
    model.add(Dropout(drop_out))
    model.add(LSTM(neurons, return_sequences=True))
    model.add(LSTM(neurons, return_sequences=False))
    model.add(Dropout(drop_out))
    model.add(Dense(dense_output, activation='linear'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
    model.predict(x_test)

    data = x_test
    prediction_seqs = []
    window_size = sequence_length
    pre_win_num = int(len(data) / prediction_len)
    for i in range(0, pre_win_num): 
        curr_frame = data[i * prediction_len]
        predicted = []
        for j in range(0, prediction_len):
            temp = model.predict(curr_frame[newaxis, :, :])[0]
            predicted.append(temp)
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size - 2], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    de_predicted = []
    len_pre_win = int(len(data) / prediction_len)
    len_pre = prediction_len
    m = 0
    for i in range(0, len_pre_win):
        for j in range(0, len_pre):
            de_predicted.append(prediction_seqs[i][j][0] * record_max[m] + record_min[m])
            m = m + 1
    error = []
    diff = y_test.shape[0] - prediction_len * pre_win_num
    for i in range(y_test_ori.shape[0] - diff):
        error.append(y_test_ori[i,] - de_predicted[i])
    squaredError = []
    for val in error:
        squaredError.append(val * val)

    MSE = sum(squaredError) / len(squaredError)
    print("*****************************************************************************************************")
    print("LSTM-MSE:{}".format(MSE))
    print("*****************************************************************************************************")

In [4]:
randomForest()
singleInputLSTM()

*******************************************************************************************
Starting date:  2017-12-07 00:00:00
Ending date:  2018-06-01 00:00:00
Duration:  176 days 00:00:00
Random forest MSE:  2546.2569412935563
*******************************************************************************************




Epoch 1/5
1/1 [==============================] - 17s 17s/step - loss: 0.4997
Epoch 2/5
1/1 [==============================] - 0s 46ms/step - loss: 0.4672
Epoch 3/5
1/1 [==============================] - 0s 57ms/step - loss: 0.4353
Epoch 4/5
1/1 [==============================] - 0s 54ms/step - loss: 0.4032
Epoch 5/5
1/1 [==============================] - 0s 62ms/step
*****************************************************************************************************
LSTM-MSE:[2121.45161221]
*****************************************************************************************************


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU
def gridSearch():
    bist100 = pd.read_csv("/content/drive/MyDrive/dp/SP500.csv")
    bist100.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close"},
                   inplace=True)
    bist100.isnull().sum()
    bist100.isna().any()
    bist100.dropna(inplace=True)
    bist100.isna().any()
    bist100['date'] = pd.to_datetime(bist100.date)
    bist100.sort_values(by='date', inplace=True)
    bist100.groupby(bist100['date'].dt.strftime('%B'))['low'].min()
    closedf = bist100[['date', 'close']]
    close_stock = closedf.copy()
    del closedf['date']
    scaler = MinMaxScaler(feature_range=(0, 1))
    closedf = scaler.fit_transform(np.array(closedf))
    
    training_size = int(len(closedf) * 0.65)
    test_size = len(closedf) - training_size
    train_data, test_data = closedf[0:test_size, :], closedf[test_size:len(closedf), :]
    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset) - time_step - 1):
            a = dataset[i:(i + time_step), 0]  
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)
    time_step = 10
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)
    from sklearn.ensemble import RandomForestRegressor
    param_test1 = [{'n_estimators':[50,120,160,200,250]},{'max_depth':[1,2,3,5,7,9,11,13]},{'min_samples_split':[100,120,150,180,200,300]}]
    gsearch1 = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_test1, cv=5)
    gsearch1.fit(X_train,y_train)
    print("*******************************************************************************************")
    print( "By gridSearch The best model is :",gsearch1.best_estimator_)
    print("*******************************************************************************************\n\n\n\n")

In [6]:
def Opt_randomForest():
    bist100 = pd.read_csv("/content/drive/MyDrive/dp/SP500.csv")
    bist100.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close"},
                   inplace=True)
    bist100.isnull().sum()
    bist100.isna().any()
    bist100.dropna(inplace=True)
    bist100.isna().any()
    bist100['date'] = pd.to_datetime(bist100.date)
    bist100.sort_values(by='date', inplace=True)
    print("*******************************************************************************************")
    print("Starting date: ", bist100.iloc[0][0])
    print("Ending date: ", bist100.iloc[-1][0])
    print("Duration: ", bist100.iloc[-1][0] - bist100.iloc[0][0])
    bist100.groupby(bist100['date'].dt.strftime('%B'))['low'].min()
    closedf = bist100[['date', 'close']]
    close_stock = closedf.copy()
    del closedf['date']
    scaler = MinMaxScaler(feature_range=(0, 1))
    closedf = scaler.fit_transform(np.array(closedf))
    training_size = int(len(closedf) * 0.65)
    test_size = len(closedf) - training_size
    train_data, test_data = closedf[0:test_size, :], closedf[test_size:len(closedf), :]
    def create_dataset(dataset, time_step=5):
        dataX, dataY = [], []
        for i in range(len(dataset) - time_step - 1):
            a = dataset[i:(i + time_step), 0] 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)

    time_step = 10
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(max_depth=11)
    regressor.fit(X_train, y_train)
    train_predict = regressor.predict(X_train)
    test_predict = regressor.predict(X_test)
    train_predict = train_predict.reshape(-1, 1)
    test_predict = test_predict.reshape(-1, 1)
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)
    original_ytrain = scaler.inverse_transform(y_train.reshape(-1, 1))
    original_ytest = scaler.inverse_transform(y_test.reshape(-1, 1))
    print("Random forest algorithm after grid search optimization MSE: ", mean_squared_error(original_ytest, test_predict))
    print("*******************************************************************************************\n\n\n\n")

In [7]:
gridSearch()

*******************************************************************************************
By grindSearch The best model is : RandomForestRegressor(max_depth=11)
*******************************************************************************************






In [8]:
randomForest()

Opt_randomForest()


*******************************************************************************************
Starting date:  2017-12-07 00:00:00
Ending date:  2018-06-01 00:00:00
Duration:  176 days 00:00:00
Random forest MSE:  2439.9474727924794
*******************************************************************************************




*******************************************************************************************
Starting date:  2017-12-07 00:00:00
Ending date:  2018-06-01 00:00:00
Duration:  176 days 00:00:00
Random forest algorithm after grid search optimization MSE:  1684.2587625453582
*******************************************************************************************






In [9]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster
import statsmodels.api as sm 
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats
def add_noise():
  df = pd.read_csv("/content/drive/MyDrive/dp/source_price.csv")

  wsj_var=np.var(df.wsj_mean_compound)
  cnbc_var=np.var(df.cnbc_mean_compound)
  fortune_var=np.var(df.fortune_mean_compound)
  reuters_var=np.var(df.reuters_mean_compound)

  mu=0
  sigma_wsj=0.1*wsj_var
  sigma_cnbc=0.1*cnbc_var
  sigma_fortune=0.1*fortune_var
  sigma_reuters=0.1*reuters_var
  n=df.shape[0]
  df_noise=pd.DataFrame()
  df_noise['wsj_noise']=df['wsj_mean_compound']
  df_noise['cnbc_noise']=df['cnbc_mean_compound']
  df_noise['fortune_noise']=df['fortune_mean_compound']
  df_noise['reuters_noise']=df['reuters_mean_compound']
  for i in range(0,n):
    df_noise['wsj_noise'][i]+=np.random.normal(mu,sigma_wsj)
    df_noise['cnbc_noise'][i]+=np.random.normal(mu,sigma_cnbc)
    df_noise['fortune_noise'][i]+=np.random.normal(mu,sigma_fortune)
    df_noise['reuters_noise'][i]+=np.random.normal(mu,sigma_reuters)
  df_noise.to_csv("/content/drive/MyDrive/dp/source_price_noise.csv")
  print("*****************************************************************************************************")
  print("Gaussian noise was successfully added to the data set")
  print("*****************************************************************************************************")

In [10]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats
def twoWeekSentimentLSTM():
    df = pd.read_csv("/content/drive/MyDrive/dp/source_price.csv")
    dfn = pd.read_csv("/content/drive/MyDrive/dp/source_price_noise.csv", index_col=0)

    df_1n = pd.DataFrame()
    df_1n['wsj'] = dfn['wsj_noise']
    df_1n['cnbc'] = df['cnbc_mean_compound']
    df_1n['fortune'] = df['fortune_mean_compound']
    df_1n['reuters'] = df['reuters_mean_compound']
    df_1n['price'] = df['Adj Close']

    df_2n = pd.DataFrame()
    df_2n['wsj'] = df['wsj_mean_compound']
    df_2n['cnbc'] = dfn['cnbc_noise']
    df_2n['fortune'] = df['fortune_mean_compound']
    df_2n['reuters'] = df['reuters_mean_compound']
    df_2n['price'] = df['Adj Close']

    df_3n = pd.DataFrame()
    df_3n['wsj'] = df['wsj_mean_compound']
    df_3n['cnbc'] = df['cnbc_mean_compound']
    df_3n['fortune'] = dfn['fortune_noise']
    df_3n['reuters'] = df['reuters_mean_compound']
    df_3n['price'] = df['Adj Close']

    df_4n = pd.DataFrame()
    df_4n['wsj'] = df['wsj_mean_compound']
    df_4n['cnbc'] = df['cnbc_mean_compound']
    df_4n['fortune'] = df['fortune_mean_compound']
    df_4n['reuters'] = dfn['reuters_noise']
    df_4n['price'] = df['Adj Close']

    df1 = df_1n
    df2 = df_2n
    df3 = df_3n
    df4 = df_4n

    split = (0.65)
    sequence_length = 10;
    normalise = True
    batch_size = 100;
    input_dim = 5
    input_timesteps = 9
    neurons = 50
    epochs = 5
    prediction_len = 1
    dense_output = 1
    drop_out = 0
    i_split = int(len(df1) * split)
    cols = ['price', 'wsj', 'cnbc', 'fortune', 'reuters']
    data_train_1 = df1.get(cols).values[:i_split]
    data_train_2 = df2.get(cols).values[:i_split]
    data_train_3 = df3.get(cols).values[:i_split]
    data_train_4 = df4.get(cols).values[:i_split]
    len_train = len(data_train_1)
    len_train_windows = None
    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_1[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1): 
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max) 
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train1 = data_windows[:, :-1]
    y_train1 = data_windows[:, -1, [0]]


    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_2[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1): 
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max) 
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train2 = data_windows[:, :-1]
    y_train2 = data_windows[:, -1, [0]]

    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_3[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1): 
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min)  
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train3 = data_windows[:, :-1]
    y_train3 = data_windows[:, -1, [0]]

    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_4[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1): 
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train4 = data_windows[:, :-1]
    y_train4 = data_windows[:, -1, [0]]

    x_train_t = np.concatenate((x_train1, x_train2, x_train3, x_train4), axis=0)
    x_train = x_train_t
    y_train_t = np.concatenate((y_train1, y_train2, y_train3, y_train4), axis=0)
    y_train = y_train_t

    dataframe = pd.read_csv("/content/drive/MyDrive/dp/source_price.csv")
    dataframe.columns = ['date', 'wsj', 'cnbc', 'fortune', 'reuters', 'price']
    cols = ['price', 'wsj', 'cnbc', 'fortune', 'reuters']
    len_dataframe = dataframe.shape[0]
    i_split = int(len(dataframe) * split)
    data_test = dataframe.get(cols).values[i_split:]
    len_test = len(data_test)
    len_train_windows = None

    data_windows = []
    for i in range(len_test - sequence_length):
        data_windows.append(data_test[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    y_test_ori = data_windows[:, -1, [0]]
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []

    # normalize
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1): 
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max) 
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)

    data_windows = normalised_data 
    x_test = data_windows[:, :-1]
    y_test = data_windows[:, -1, [0]]

    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences=True))
    model.add(Dropout(drop_out))
    model.add(LSTM(neurons, return_sequences=True))
    model.add(LSTM(neurons, return_sequences=False))
    model.add(Dropout(drop_out))
    model.add(Dense(dense_output, activation='linear'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

    data = x_test
    prediction_seqs = []
    window_size = sequence_length
    pre_win_num = int(len(data) / prediction_len)
    for i in range(0, pre_win_num):
        curr_frame = data[i * prediction_len]
        predicted = []
        for j in range(0, prediction_len):
            temp = model.predict(curr_frame[newaxis, :, :])[0]
            predicted.append(temp)
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size - 2], predicted[-1], axis=0)
        prediction_seqs.append(predicted)

    de_predicted = []
    len_pre_win = int(len(data) / prediction_len)
    len_pre = prediction_len
    m = 0
    for i in range(0, len_pre_win):
        for j in range(0, len_pre):
            de_predicted.append(prediction_seqs[i][j][0] * record_max[m] + record_min[m])
            m = m + 1
    error = []
    diff = y_test.shape[0] - prediction_len * pre_win_num
    for i in range(y_test_ori.shape[0] - diff):
        error.append(y_test_ori[i,] - de_predicted[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)

    MSE = sum(squaredError) / len(squaredError)
    print("*****************************************************************************************************")
    print("Sentimental-LSTM MSE:{}".format(MSE))
    print("*****************************************************************************************************")

In [11]:
dd = pd.read_csv("/content/drive/MyDrive/dp/source_price.csv")
dd.head()

,date,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,Adj Close
0,2017/12/7,0.296,-0.1366,0.0000,0.0,2636.979980
1,2017/12/8,0.000,0.0000,-0.2423,0.0,2651.500000
2,2017/12/11,0.000,0.0000,0.0000,0.0,2659.989990
3,2017/12/12,0.000,0.0000,0.0000,0.0,2664.110107
4,2017/12/13,0.000,0.0000,0.0000,0.0,2662.850098


In [12]:
add_noise()

*****************************************************************************************************
Gaussian noise was successfully added to the data set
*****************************************************************************************************


In [13]:
singleInputLSTM()
twoWeekSentimentLSTM()

Epoch 1/5
1/1 [==============================] - 6s 6s/step - loss: 0.5312
Epoch 2/5
1/1 [==============================] - 0s 30ms/step - loss: 0.4975
Epoch 3/5
1/1 [==============================] - 0s 46ms/step - loss: 0.4667
Epoch 4/5
1/1 [==============================] - 0s 32ms/step - loss: 0.4377
Epoch 5/5
1/1 [==============================] - 0s 54ms/step
*****************************************************************************************************
LSTM-MSE:[2429.24941014]
*****************************************************************************************************
Epoch 1/5
3/3 [==============================] - 7s 28ms/step - loss: 0.4886
Epoch 2/5
3/3 [==============================] - 0s 25ms/step - loss: 0.3926
Epoch 3/5
3/3 [==============================] - 0s 26ms/step - loss: 0.2987
Epoch 4/5
3/3 [==============================] - 0s 28ms/step - loss: 0.2212
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
*********************************

In [14]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats
def oneWeekSentimentLSTM():
    df = pd.read_csv("/content/drive/MyDrive/dp/source_price.csv")
    dfn = pd.read_csv("/content/drive/MyDrive/dp/source_price_noise.csv", index_col=0)

    df_1n = pd.DataFrame()
    df_1n['wsj'] = dfn['wsj_noise']
    df_1n['cnbc'] = df['cnbc_mean_compound']
    df_1n['fortune'] = df['fortune_mean_compound']
    df_1n['reuters'] = df['reuters_mean_compound']
    df_1n['price'] = df['Adj Close']

    df_2n = pd.DataFrame()
    df_2n['wsj'] = df['wsj_mean_compound']
    df_2n['cnbc'] = dfn['cnbc_noise']
    df_2n['fortune'] = df['fortune_mean_compound']
    df_2n['reuters'] = df['reuters_mean_compound']
    df_2n['price'] = df['Adj Close']

    df_3n = pd.DataFrame()
    df_3n['wsj'] = df['wsj_mean_compound']
    df_3n['cnbc'] = df['cnbc_mean_compound']
    df_3n['fortune'] = dfn['fortune_noise']
    df_3n['reuters'] = df['reuters_mean_compound']
    df_3n['price'] = df['Adj Close']

    df_4n = pd.DataFrame()
    df_4n['wsj'] = df['wsj_mean_compound']
    df_4n['cnbc'] = df['cnbc_mean_compound']
    df_4n['fortune'] = df['fortune_mean_compound']
    df_4n['reuters'] = dfn['reuters_noise']
    df_4n['price'] = df['Adj Close']

    df1 = df_1n
    df2 = df_2n
    df3 = df_3n
    df4 = df_4n

    split = (0.65)
    sequence_length = 5;
    normalise = True
    batch_size = 100;
    input_dim = 5
    input_timesteps = 4
    neurons = 50
    epochs = 5
    prediction_len = 1
    dense_output = 1
    drop_out = 0

    i_split = int(len(df1) * split)
    cols = ['price', 'wsj', 'cnbc', 'fortune', 'reuters']
    data_train_1 = df1.get(cols).values[:i_split]
    data_train_2 = df2.get(cols).values[:i_split]
    data_train_3 = df3.get(cols).values[:i_split]
    data_train_4 = df4.get(cols).values[:i_split]
    len_train = len(data_train_1)
    len_train_windows = None
    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_1[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1):  
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train1 = data_windows[:, :-1]
    y_train1 = data_windows[:, -1, [0]]


    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_2[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1): 
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max) 
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train2 = data_windows[:, :-1]
    y_train2 = data_windows[:, -1, [0]]

    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_3[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1):  
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min)  
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max) 
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train3 = data_windows[:, :-1]
    y_train3 = data_windows[:, -1, [0]]

    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train_4[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []
    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1): 
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)
    data_windows = normalised_data
    x_train4 = data_windows[:, :-1]
    y_train4 = data_windows[:, -1, [0]]

    x_train_t = np.concatenate((x_train1, x_train2, x_train3, x_train4), axis=0)
    x_train = x_train_t
    y_train_t = np.concatenate((y_train1, y_train2, y_train3, y_train4), axis=0)
    y_train = y_train_t

    dataframe = pd.read_csv("/content/drive/MyDrive/dp/source_price.csv")
    dataframe.columns = ['date', 'wsj', 'cnbc', 'fortune', 'reuters', 'price']
    cols = ['price', 'wsj', 'cnbc', 'fortune', 'reuters']
    len_dataframe = dataframe.shape[0]
    i_split = int(len(dataframe) * split)
    data_test = dataframe.get(cols).values[i_split:]
    len_test = len(data_test)
    len_train_windows = None

    data_windows = []
    for i in range(len_test - sequence_length):
        data_windows.append(data_test[i:i + sequence_length])
    data_windows = np.array(data_windows).astype(float)
    y_test_ori = data_windows[:, -1, [0]]

    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]
    normalised_data = []
    record_min = []
    record_max = []

    for win_i in range(0, win_num):
        normalised_window = []
        for col_i in range(0, 1):
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0:
                record_min.append(temp_min) 
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)  
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        for col_i in range(1, col_num):
            temp_col = window_data[win_i, :, col_i]
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data = np.array(normalised_data)

    data_windows = normalised_data 
    x_test = data_windows[:, :-1]
    y_test = data_windows[:, -1, [0]]
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences=True))
    model.add(Dropout(drop_out))
    model.add(LSTM(neurons, return_sequences=True))
    model.add(LSTM(neurons, return_sequences=False))
    model.add(Dropout(drop_out))
    model.add(Dense(dense_output, activation='linear'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

    data = x_test
    prediction_seqs = []
    window_size = sequence_length
    pre_win_num = int(len(data) / prediction_len)
    for i in range(0, pre_win_num):
        curr_frame = data[i * prediction_len]
        predicted = []
        for j in range(0, prediction_len):
            temp = model.predict(curr_frame[newaxis, :, :])[0]
            predicted.append(temp)
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size - 2], predicted[-1], axis=0)
        prediction_seqs.append(predicted)

    de_predicted = []
    len_pre_win = int(len(data) / prediction_len)
    len_pre = prediction_len
    m = 0
    for i in range(0, len_pre_win):
        for j in range(0, len_pre):
            de_predicted.append(prediction_seqs[i][j][0] * record_max[m] + record_min[m])
            m = m + 1
    error = []
    diff = y_test.shape[0] - prediction_len * pre_win_num
    for i in range(y_test_ori.shape[0] - diff):
        error.append(y_test_ori[i,] - de_predicted[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)

    MSE = sum(squaredError) / len(squaredError)
    print("*****************************************************************************************************")
    print("one week Sentimental-LSTM MSE:{}".format(MSE))
    print("*****************************************************************************************************")

In [15]:
twoWeekSentimentLSTM()
oneWeekSentimentLSTM()

Epoch 1/5
3/3 [==============================] - 7s 29ms/step - loss: 0.4675
Epoch 2/5
3/3 [==============================] - 0s 29ms/step - loss: 0.3699
Epoch 3/5
3/3 [==============================] - 0s 27ms/step - loss: 0.2723
Epoch 4/5
3/3 [==============================] - 0s 29ms/step - loss: 0.2211
Epoch 5/5
1/1 [==============================] - 0s 23ms/step
*****************************************************************************************************
Sentimental-LSTM MSE:[826.27159857]
*****************************************************************************************************
Epoch 1/5
3/3 [==============================] - 8s 16ms/step - loss: 0.5012
Epoch 2/5
3/3 [==============================] - 0s 17ms/step - loss: 0.4610
Epoch 3/5
3/3 [==============================] - 0s 16ms/step - loss: 0.4195
Epoch 4/5
3/3 [==============================] - 0s 16ms/step - loss: 0.3729
Epoch 5/5
1/1 [==============================] - 0s 25ms/step
********************